In [3]:

import torch
import jovian
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

PyTorch nima?
PyTorch Python kutubxonasi boʻlib, chuqur oʻrganish modellarini yaratish vositalarini taqdim etadi. PyTorch dasturlash uchun python nima qiladi, chuqur o'rganish uchun nima qiladi. Python dasturlash uchun juda moslashuvchan tildir va xuddi python kabi PyTorch kutubxonasi chuqur o'rganish uchun moslashuvchan vositalarni taqdim etadi. Agar siz chuqur o'rganishni o'rganayotgan bo'lsangiz yoki u bilan boshlashni istasangiz, unda PyTorch bilimi sizga chuqur o'rganish modellarini yaratishda katta yordam beradi.

Shuningdek, o'qing - Mashinani o'rganish uchun matematika.

PyTorch yordamida avtomobil narxlarini taxmin qiling
Keling, PyTorch yordamida avtomobil narxlarini bashorat qilishni mashinani o'rganish vazifasidan boshlaylik. Men ushbu vazifa uchun zarur bo'lgan barcha kutubxonalarni import qilishdan boshlayman. Men ushbu vazifada foydalanadigan ma'lumotlar to'plamini bu yerdan osongina yuklab olish mumkin :

In [2]:
!pip install jovian

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 6.1 MB/s 
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6503 sha256=bb7a0ded2097b1658ac232b258cde2e7fe4ba2abd76517e06d581a5402463568
  Stored in directory: /root/.cache/pip/wheels/2a/ea/87/dd57f1ecb4f0752f3e1dbf958ebf8b36d920d190425bcdc24d
Successfully built uuid


In [5]:
DATA_FILENAME = "/content/car_data.csv"
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


Siz ma'lumotlar qanday ko'rinishini ko'rishingiz mumkin, lekin uni ishlatishdan oldin biz uni sozlashimiz, o'qlarni tartiblashimiz va bashorat qilishga yordam bermaydigan ustunlarni olib tashlashimiz kerak, bu erda biz avtomobil nomlarini tushiramiz va bu moslashtirishni amalga oshirish uchun biz quyidagi funksiya:

In [6]:
your_name = "Aman Kharwal" # at least 5 characters
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.Year = dataframe.Year * ord(rand_str[1])/100.
    # scale target
    dataframe.Selling_Price = dataframe.Selling_Price * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['Car_Name'], axis=1)
    return dataframe

dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
2,ciaz,2198.53,7.0325,9.85,6900,Petrol,Dealer,Manual,0
128,Honda CB Hornet 160R,2198.53,0.7760,0.87,3000,Petrol,Individual,Manual,0
286,jazz,2197.44,5.4805,7.90,28569,Petrol,Dealer,Manual,0
272,city,2196.35,7.2750,10.00,27600,Petrol,Dealer,Manual,0
205,grand i10,2197.44,5.0925,5.70,3493,Petrol,Dealer,Manual,1


Yuqoridagi ushbu funktsiyada ma'lumotlarni tasodifiy tartiblash uchun tasodifiy qator sifatida foydalanish uchun so'z kerakligini ko'rib turibmiz, men o'z ismimni tasodifiy qator sifatida ishlatganman. Shundan so'ng biz maxsus ma'lumotlar to'plamidan foydalanishimiz mumkin, soddaligi uchun biz raqamli, toifali yoki chiqish ustunlarini o'z ichiga olgan qatorlar, ustunlar va o'zgaruvchilar sonini o'z ichiga olgan o'zgaruvchilarni yaratishimiz mumkin:

In [9]:
input_cols = ["Year","Present_Price","Kms_Driven","Owner"]
categorical_cols = ["Fuel_Type","Seller_Type","Transmission"]
output_cols = ["Selling_Price"]

Ma'lumotlarni tayyorlash
Maqolaning boshida aytib o'tilganidek, men PyTorch-dan mashinalarni o'rganishdan foydalangan holda avtomobil narxlarini bashorat qilish uchun foydalanaman, shuning uchun ma'lumotni o'qitish uchun ishlatish uchun biz uni dataframe'dan PyTorch Tensorlariga aylantirishimiz kerak, birinchi qadam NumPy massivlariga aylantirishdir:

In [10]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[2.19853e+03, 9.85000e+00, 6.90000e+03, 0.00000e+00],
        [2.19853e+03, 8.70000e-01, 3.00000e+03, 0.00000e+00],
        [2.19744e+03, 7.90000e+00, 2.85690e+04, 0.00000e+00],
        ...,
        [2.19853e+03, 3.62300e+01, 6.00000e+03, 0.00000e+00],
        [2.18872e+03, 5.20000e-01, 5.00000e+05, 0.00000e+00],
        [2.19635e+03, 8.40000e-01, 5.80000e+04, 0.00000e+00]]),
 array([[ 7.0325],
        [ 0.776 ],
        [ 5.4805],
        [ 7.275 ],
        [ 5.0925],
        [ 0.4656],
        [ 2.7645],
        [ 0.3395],
        [ 1.6975],
        [ 4.8015],
        [ 8.9725],
        [ 5.141 ],
        [ 3.783 ],
        [ 8.827 ],
        [ 4.268 ],
        [ 3.3853],
        [19.1575],
        [ 7.0325],
        [10.9125],
        [12.513 ],
        [14.2881],
        [ 4.753 ],
        [ 5.0925],
        [ 3.1525],
        [ 7.2265],
        [ 0.5335],
        [ 2.813 ],
        [ 0.3007],
        [ 0.2619],
        [ 7.178 ],
        [ 0.582 ],
        [ 5.9655],
     

Yuqoridagi funktsiya kirish va chiqish ustunlarini NumPy massivlariga o'zgartiradi, tekshirish uchun natijani ko'rsatish mumkin va siz ma'lumotlarning massivlarga qanday aylanishini ko'rishingiz mumkin. Endi ushbu massivlarga ega bo'lgan holda, biz ularni PyTorch tensorlariga aylantirishimiz va ularni o'z ichiga olgan o'zgaruvchan ma'lumotlar to'plamini yaratish uchun ushbu tensorlardan foydalanishimiz mumkin:

In [11]:
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

dataset = TensorDataset(inputs, targets)
train_ds, val_ds = random_split(dataset, [228, 57])
batch_size = 128

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

PyTorch modelini yaratish
Endi men avtomobil narxlarini bashorat qilish uchun PyTorch yordamida chiziqli regressiya modelini yaratmoqchiman:

In [12]:
input_size = len(input_cols)
output_size = len(output_cols)

class CarsModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)                         # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))
            
model = CarsModel()

list(model.parameters())

[Parameter containing:
 tensor([[-0.3751,  0.3582,  0.4767,  0.1272]], requires_grad=True),
 Parameter containing:
 tensor([0.3690], requires_grad=True)]

Ushbu yuqoridagi funktsiyada men nn.Linear funksiyasidan foydalandim, bu bizga chiziqli regressiyadan foydalanish imkonini beradi, shuning uchun endi biz bashoratlarni hisoblashimiz mumkin va F.l1_loss funktsiyasi yordamida yo'qotish vazn parametrini bir taraflama ko'rishimiz mumkin, bu model bilan biz olamiz bashoratlar, lekin hali ham treningdan o'tishlari kerak.

Avtomobil narxlarini bashorat qilish uchun o'quv modeli
Endi biz yo'qotishni baholab, qancha ekanligini ko'rishimiz kerak va mashg'ulotdan so'ng mashg'ulot bilan yo'qotish qanchalik kamayishini ko'rishimiz kerak

In [13]:
# Eval algorithm
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

# Fitting algorithm
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

# Check the initial value that val_loss have
result = evaluate(model, val_loader)
print(result)

{'val_loss': 20590.56640625}


In [14]:

# Start with the Fitting
epochs = 90
lr = 1e-8
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 19952.0508
Epoch [40], val_loss: 19315.0469
Epoch [60], val_loss: 18677.9414
Epoch [80], val_loss: 18040.3516
Epoch [90], val_loss: 17722.0566


In [15]:

# Train repeatdly until have a 'good' val_loss
epochs = 20
lr = 1e-9
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 17658.4414


Ko'rib turganingizdek, baholash va moslashtirish uchun model funktsiyalaridan foydalaniladi, mashg'ulotlarni o'tkazish uchun biz optimallashtirish funktsiyalaridan foydalanamiz, bu holda, xususan, SGD optimallashtirish, poezd yuklagichi yordamida yo'qotish va gradientlarni hisoblab chiqamiz, keyin uni optimallashtiramiz va har bir iteratsiya natijasini baholaymiz. yo'qotishni ko'ring.


Avtomobil narxlarini bashorat qilish uchun modeldan foydalanish
Nihoyat, biz modelni aniq ma'lumotlar bilan sinab ko'rishimiz kerak, bashorat qilish uchun biz ma'lumotlar to'plamida ko'radigan kirish qiymatlari bo'ladigan kirishdan foydalanishimiz kerak va model o'tish uchun biz qiladigan Avtomobillar modelidir. modelda tekislash kerak, shuning uchun bularning barchasi bilan sotish narxlarini taxmin qiling:

In [16]:
# Prediction Algorithm
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

# Testing the model with some samples
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([2.1974e+03, 7.8700e+00, 2.4524e+04, 0.0000e+00])
Target: tensor([5.6745])
Prediction: tensor([9253.6562])


Ko'rib turganingizdek, bashoratlar kutilgan maqsadga juda yaqin, aniq emas, lekin kutilganiga o'xshash. Bu bilan endi turli natijalarni sinab ko'rish va model qanchalik yaxshi ekanligini ko'rish mumkin: